# Máster en Data Science - Machine Learning

# Procesamiento de variables: Codificación de variables categóricas y escalado
Autor: Ramón Morillo Barrera

## Dataset: Application data

En este notebook trabajaremos en el procesamiento de nuestras variables categóricas del Dataset, con el objetivo de que queden listas para ser procesadas por los modelos que probaremos.

Aplicaré diferentes técnicas de encoding que transforman las variables categóricas de nuestro Dataset a numéricas y posteriormente permitirán a los modelos que implementemos procesar las variables categóricas presentes en nuestro Dataset. Además de un escalado de variables.

El escalado de variables es un proceso de transformar los datos para que todas las características estén en la misma escala, generalmente normalizando los valores dentro de un rango (por ejemplo, 0 a 1) o estandarizándolos para que tengan media 0 y desviación estándar 1. Esto evita que las variables con mayores rangos predominen en el modelo o afecten a los cálculos del mismo, especialmente en algoritmos basados en distancia (como SVM, KNN, PCA) o gradientes (como redes neuronales). Además, mejora el desempeño ya que haace que algoritmos sensibles a la magnitud de las variables (por ejemplo, KNN) funcionen mejor y evita sesgos haciendo que las variables con grandes rangos no tendrán mayor peso que otras menos dispersas.

#### Librerías

In [1]:
import sys
import pandas as pd 
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics
# conda install category_encoders
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve, accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix

from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay

#### Funciones

In [2]:
sys.path.append('../src')
import funciones_auxiliares as f_aux
sys.path.remove('../src')

# Constante
seed = 12354

#### Importación de datos

In [3]:
df_loan_train = pd.read_csv("../../data_loan_status/data_split/df_loan_train.csv")
df_loan_test = pd.read_csv("../../data_loan_status/data_split/df_loan_test.csv")

In [4]:
df_loan_train.head()

,SK_ID_CURR,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,FONDKAPREMONT_MODE,LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_AVG,...,FLAG_DOCUMENT_13,FLAG_DOCUMENT_19,FLAG_DOCUMENT_18,FLAG_DOCUMENT_17,FLAG_DOCUMENT_16,FLAG_DOCUMENT_15,FLAG_DOCUMENT_14,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,TARGET
0,192624,0.0276,0.0278,0.0279,0.0039,0.0039,0.0039,reg oper account,0.1223,0.1202,...,0,0,0,0,0,0,0,0,0,0
1,166840,0.0466,0.0469,0.0470,0.0000,0.0000,0.0000,not specified,0.0778,0.0765,...,0,0,0,0,0,0,0,0,0,0
2,428919,0.0211,0.0208,0.0190,0.0000,0.0000,0.0000,Desconocido,0.0761,0.0756,...,0,0,0,0,0,0,0,0,0,0
3,138440,0.0211,0.0208,0.0190,0.0000,0.0000,0.0000,Desconocido,0.0761,0.0756,...,0,0,0,0,0,0,0,0,0,0
4,155286,0.0211,0.0208,0.0190,0.0000,0.0000,0.0000,Desconocido,0.0761,0.0756,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_loan_train.columns

Index(['SK_ID_CURR', 'COMMONAREA_AVG', 'COMMONAREA_MEDI', 'COMMONAREA_MODE',
       'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAPARTMENTS_MEDI',
       'NONLIVINGAPARTMENTS_MODE', 'FONDKAPREMONT_MODE',
       'LIVINGAPARTMENTS_MEDI', 'LIVINGAPARTMENTS_AVG',
       ...
       'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_18',
       'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_15',
       'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'TARGET'],
      dtype='object', length=122)

In [6]:
df_loan_train.head()

,SK_ID_CURR,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,FONDKAPREMONT_MODE,LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_AVG,...,FLAG_DOCUMENT_13,FLAG_DOCUMENT_19,FLAG_DOCUMENT_18,FLAG_DOCUMENT_17,FLAG_DOCUMENT_16,FLAG_DOCUMENT_15,FLAG_DOCUMENT_14,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,TARGET
0,192624,0.0276,0.0278,0.0279,0.0039,0.0039,0.0039,reg oper account,0.1223,0.1202,...,0,0,0,0,0,0,0,0,0,0
1,166840,0.0466,0.0469,0.0470,0.0000,0.0000,0.0000,not specified,0.0778,0.0765,...,0,0,0,0,0,0,0,0,0,0
2,428919,0.0211,0.0208,0.0190,0.0000,0.0000,0.0000,Desconocido,0.0761,0.0756,...,0,0,0,0,0,0,0,0,0,0
3,138440,0.0211,0.0208,0.0190,0.0000,0.0000,0.0000,Desconocido,0.0761,0.0756,...,0,0,0,0,0,0,0,0,0,0
4,155286,0.0211,0.0208,0.0190,0.0000,0.0000,0.0000,Desconocido,0.0761,0.0756,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df_loan_train.describe()

,SK_ID_CURR,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_MODE,...,FLAG_DOCUMENT_13,FLAG_DOCUMENT_19,FLAG_DOCUMENT_18,FLAG_DOCUMENT_17,FLAG_DOCUMENT_16,FLAG_DOCUMENT_15,FLAG_DOCUMENT_14,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,TARGET
count,246008.000000,246008.000000,246008.000000,246008.000000,246008.000000,246008.000000,246008.000000,246008.000000,246008.000000,246008.000000,...,246008.000000,246008.000000,246008.000000,246008.000000,246008.000000,246008.000000,246008.000000,246008.000000,246008.000000,246008.000000
mean,278263.240862,0.028107,0.027889,0.026013,0.002686,0.002640,0.002464,0.084194,0.083485,0.085432,...,0.003561,0.000569,0.008207,0.000244,0.009983,0.001236,0.002988,0.000504,0.000329,0.080729
std,102775.595271,0.042584,0.042684,0.041758,0.026572,0.026437,0.025737,0.053732,0.053106,0.056453,...,0.059567,0.023849,0.090220,0.015615,0.099417,0.035131,0.054578,0.022445,0.018143,0.272419
min,100007.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189262.750000,0.021100,0.020800,0.019000,0.000000,0.000000,0.000000,0.076100,0.075600,0.076200,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278295.500000,0.021100,0.020800,0.019000,0.000000,0.000000,0.000000,0.076100,0.075600,0.076200,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,367235.250000,0.021100,0.020800,0.019000,0.000000,0.000000,0.000000,0.076100,0.075600,0.076200,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,456255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Realizamos una pequeña verificación para ver que los datos se importaron en el formato correcto, al parecer esta todo bien.

## Codificación de variables categoricas

La variable objetivo no tendremos que codificarla, pues ya toma el valor [0, 1]. A continuación voy a desarrollar y proporcionar una breve definición de los diferentes tipos de codificación de variables que he encontrado. Posteriormente realizaré un pequeño estudio de mis variables categóricas para ver y definir cuántas categorías presenta cada una, para, finalmente decidir que tipo de codificación aplicar a cada variable.

- **One-Hot encoding**: Convierte cada valor único de una variable categórica en una nueva columna binaria (0 o 1), donde un valor específico se marca con 1 y el resto con 0. Utilizado cuando no hay un orden natural entre las categorías y no queremos que el modelo asuma una relación jerárquica entre ellas.

- **Mean encoding**: Reemplaza cada valor de la variable categórica por la media del target (variable dependiente) correspondiente a esa categoría. Es útil cuando hay una relación estadística entre la variable categórica y la variable objetivo, pero puede causar overfitting si no se maneja correctamente.

- **Ordinal encoding**: Asigna un valor numérico a cada categoría de una variable categórica, respetando el orden natural de las categorías. Es ideal cuando existe un orden implícito entre las categorías y el modelo puede aprovechar esa relación.

- **Target Encoding**: Similar al mean encoding, pero en lugar de usar la media global de la variable, utiliza el promedio de la variable objetivo por categoría. Proporciona un valor más relevante y específico para cada categoría al incorporar la información de la variable objetivo, pero debe ser utilizado cuidadosamente para evitar el sobreajuste.

- **CatBoost Encoding**: Método especializado en codificación para modelos de CatBoost, que se basa en la codificación por target encoding, pero ajusta el valor usando un proceso de suavizado y utilizando la frecuencia de las categorías. Es especialmente útil en problemas con variables categóricas de alta cardinalidad y mejora la precisión de modelos que utilizan árboles de decisión, como CatBoost, LightGBM y XGBoost.

In [8]:
cat_vars = df_loan_train.select_dtypes(include=['object']).columns

# Contar valores únicos en cada variable categórica
unique_counts = df_loan_train[cat_vars].nunique()

print(unique_counts)

FONDKAPREMONT_MODE             5
WALLSMATERIAL_MODE             8
HOUSETYPE_MODE                 4
EMERGENCYSTATE_MODE            3
OCCUPATION_TYPE               19
NAME_TYPE_SUITE                8
ORGANIZATION_TYPE             58
NAME_CONTRACT_TYPE             2
FLAG_OWN_CAR                   2
CODE_GENDER                    3
NAME_INCOME_TYPE               8
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
NAME_EDUCATION_TYPE            5
FLAG_OWN_REALTY                2
WEEKDAY_APPR_PROCESS_START     7
dtype: int64


Después de observar el número de valores únicos que presentan nuestras variables categóricas. Voy a establecer los siguientes criterios de encoding:
- Baja cardinalidad (< 10 categorías):
Usar One-Hot Encoding.
- Media/Alta cardinalidad (10-50 categorías):
Considerar Target Encoding o Ordinal Encoding (si hay orden implícito). En este caso como no hay orden implícito, usaré Target Encoding.
- Alta cardinalidad (> 50 categorías):
Usar técnicas específicas como Mean Encoding, CatBoost Encoding, para reducir dimensionalidad y evitar overfitting. En este caso usaré CatBoost Encoding.

#### Separación X e y

In [9]:
y_train = df_loan_train['TARGET']
X_train = df_loan_train.drop('TARGET', axis=1)
y_test = df_loan_test['TARGET']
X_test = df_loan_test.drop('TARGET', axis=1)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((246008, 121), (246008,), (61503, 121), (61503,))

### 1. One-Hot Encoding

In [88]:
list_columns_cat = list(df_loan_train.select_dtypes(include=["object", "category"]).columns)
exclude_vars = ['OCCUPATION_TYPE', 'ORGANIZATION_TYPE']  # Excluir estas columnas
list_columns_ohe = [col for col in list_columns_cat if col not in exclude_vars]

# Crear y aplicar One-Hot Encoder
ohe = ce.OneHotEncoder(cols=list_columns_ohe, use_cat_names=True)
ohe.fit(X_train, y_train)  # Ajustar solo con los datos de entrenamiento

# Transformar X_train y X_test
X_train_t = ohe.transform(X_train)
X_test_t = ohe.transform(X_test)

# Verificar formas finales
print(X_train_t.shape, X_test_t.shape)

(246008, 176) (61503, 176)


### 2. Target Encoding

In [96]:
# Target Encoding

target_column = 'OCCUPATION_TYPE'

# Crear y ajustar el codificador de Target Encoding
target_enc = ce.TargetEncoder(cols=[target_column])
target_enc.fit(X_train_t[target_column], y_train)  # Ajuste solo con los datos de entrenamiento

# Transformar X_train y X_test
X_train_te = X_train_t.copy()
X_test_te = X_test_t.copy()

X_train_te[target_column] = target_enc.transform(X_train_t[target_column])
X_test_te[target_column] = target_enc.transform(X_test_t[target_column])

# Verificar formas finales
print(X_train_te.shape, X_test_te.shape)


(246008, 176) (61503, 176)


### 3. CatBoost Encoding

In [97]:
# CatBoost Encoding

target_column = 'ORGANIZATION_TYPE'

# Crear y ajustar el codificador de CatBoost Encoding
catboost_enc = ce.CatBoostEncoder(cols=[target_column])
catboost_enc.fit(X_train_te[target_column], y_train)  # Ajustar solo con los datos de entrenamiento

# Transformar X_train y X_test
X_train_tec = X_train_te.copy()
X_test_tec = X_test_te.copy()

X_train_tec[target_column] = catboost_enc.transform(X_train_te[target_column])
X_test_tec[target_column] = catboost_enc.transform(X_test_te[target_column])

# Verificar formas finales
print(X_train_tec.shape, X_test_tec.shape)

(246008, 176) (61503, 176)


In [98]:
X_train_tec.dtypes.to_dict()

{'SK_ID_CURR': dtype('int64'),
 'COMMONAREA_AVG': dtype('float64'),
 'COMMONAREA_MEDI': dtype('float64'),
 'COMMONAREA_MODE': dtype('float64'),
 'NONLIVINGAPARTMENTS_AVG': dtype('float64'),
 'NONLIVINGAPARTMENTS_MEDI': dtype('float64'),
 'NONLIVINGAPARTMENTS_MODE': dtype('float64'),
 'FONDKAPREMONT_MODE_reg oper account': dtype('int64'),
 'FONDKAPREMONT_MODE_not specified': dtype('int64'),
 'FONDKAPREMONT_MODE_Desconocido': dtype('int64'),
 'FONDKAPREMONT_MODE_reg oper spec account': dtype('int64'),
 'FONDKAPREMONT_MODE_org spec account': dtype('int64'),
 'LIVINGAPARTMENTS_MEDI': dtype('float64'),
 'LIVINGAPARTMENTS_AVG': dtype('float64'),
 'LIVINGAPARTMENTS_MODE': dtype('float64'),
 'FLOORSMIN_MODE': dtype('float64'),
 'FLOORSMIN_AVG': dtype('float64'),
 'FLOORSMIN_MEDI': dtype('float64'),
 'YEARS_BUILD_MODE': dtype('float64'),
 'YEARS_BUILD_MEDI': dtype('float64'),
 'YEARS_BUILD_AVG': dtype('float64'),
 'OWN_CAR_AGE': dtype('float64'),
 'LANDAREA_MEDI': dtype('float64'),
 'LANDAREA_A

In [99]:
X_train_tec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246008 entries, 0 to 246007
Columns: 176 entries, SK_ID_CURR to FLAG_DOCUMENT_21
dtypes: float64(67), int64(109)
memory usage: 330.3 MB


In [100]:
X_train_tec.shape

(246008, 176)

Después de aplicar los encodings definidos, nuestro dataset tiene muy buena pinta. Finalmente tenemos 176 columnas todas numéricas con 246008 registros para cada columna.

Ya están listos para realizar el escalado de variables.

## Escalado de Variables

El escalado de variables es el proceso de transformar las características numéricas de un conjunto de datos para que se encuentren dentro de un rango determinado (como 0-1 o con media 0 y desviación estándar 1). Esto no altera la distribución relativa de los datos, sino que los ajusta a una escala común.

Escalar datos es esencial para evitar que algunas características dominen a otras debido a diferencias en sus magnitudes. Esto asegura que los modelos se entrenen de manera equilibrada y robusta, resultando en un mejor rendimiento y generalización.

Los modelos basados en distancia necesitan un escalado de datos porque la distancia entre puntos en el espacio de características (como la distancia Euclídea) es sensible a las magnitudes de las variables. Si las características tienen rangos diferentes, las variables con valores más grandes dominarán el cálculo de las distancias, lo que puede llevar a resultados sesgados o incorrectos.

Algoritmos sensibles a escalas de datos son:
- **K-Nearest Neighbors (KNN)**: Los vecinos más cercanos se identifican usando distancias, y un mal escalado puede llevar a clasificaciones erróneas.
- **Clustering (K-Means, DBSCAN)**: Las asignaciones de clústeres dependen de las distancias entre puntos.
- **Support Vector Machines (SVM)**: El margen de separación se define en función de distancias en el espacio transformado.

In [101]:
scaler = StandardScaler()
model_scaled = scaler.fit(X_train_tec)
X_train_scaled = pd.DataFrame(scaler.transform(X_train_tec), columns=X_train_tec.columns, index=X_train_tec.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_tec), columns=X_test_tec.columns, index=X_test_tec.index)

In [102]:
X_train_scaled.head()

,SK_ID_CURR,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_Desconocido,...,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_19,FLAG_DOCUMENT_18,FLAG_DOCUMENT_17,FLAG_DOCUMENT_16,FLAG_DOCUMENT_15,FLAG_DOCUMENT_14,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
0,-0.833266,-0.011918,-0.002080,0.045196,0.045686,0.047676,0.055811,1.777491,-0.137304,-1.470338,...,-0.002851,-0.059779,-0.023862,-0.090967,-0.015619,-0.10042,-0.035175,-0.054742,-0.022457,-0.018148
1,-1.084143,0.434262,0.445394,0.502593,-0.101085,-0.099845,-0.095722,-0.562591,7.283127,-1.470338,...,-0.002851,-0.059779,-0.023862,-0.090967,-0.015619,-0.10042,-0.035175,-0.054742,-0.022457,-0.018148
2,1.465874,-0.164558,-0.166075,-0.167936,-0.101085,-0.099845,-0.095722,-0.562591,-0.137304,0.680116,...,-0.002851,-0.059779,-0.023862,-0.090967,-0.015619,-0.10042,-0.035175,-0.054742,-0.022457,-0.018148
3,-1.360474,-0.164558,-0.166075,-0.167936,-0.101085,-0.099845,-0.095722,-0.562591,-0.137304,0.680116,...,-0.002851,-0.059779,-0.023862,-0.090967,-0.015619,-0.10042,-0.035175,-0.054742,-0.022457,-0.018148
4,-1.196563,-0.164558,-0.166075,-0.167936,-0.101085,-0.099845,-0.095722,-0.562591,-0.137304,0.680116,...,-0.002851,-0.059779,-0.023862,-0.090967,-0.015619,-0.10042,-0.035175,-0.054742,-0.022457,-0.018148


In [103]:
X_train_scaled.describe()

,SK_ID_CURR,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_Desconocido,...,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_19,FLAG_DOCUMENT_18,FLAG_DOCUMENT_17,FLAG_DOCUMENT_16,FLAG_DOCUMENT_15,FLAG_DOCUMENT_14,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
count,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,...,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05
mean,1.050580e-16,-6.401897e-17,1.515486e-16,-3.355472e-17,-1.623581e-17,-3.101303e-18,-2.570399e-17,-5.040068e-17,1.152067e-17,-4.886989e-17,...,-5.271131e-19,3.947933e-17,8.722639e-18,-4.351933e-17,-4.043608e-18,-4.603936e-17,-7.437350e-19,-1.926490e-17,8.654042e-18,1.646326e-18
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,...,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-1.734425e+00,-6.600520e-01,-6.533767e-01,-6.229374e-01,-1.010855e-01,-9.984496e-02,-9.572186e-02,-5.625907e-01,-1.373037e-01,-1.470338e+00,...,-2.851295e-03,-5.977947e-02,-2.386234e-02,-9.096682e-02,-1.561903e-02,-1.004196e-01,-3.517472e-02,-5.474177e-02,-2.245669e-02,-1.814844e-02
25%,-8.659709e-01,-1.645579e-01,-1.660753e-01,-1.679358e-01,-1.010855e-01,-9.984496e-02,-9.572186e-02,-5.625907e-01,-1.373037e-01,-1.470338e+00,...,-2.851295e-03,-5.977947e-02,-2.386234e-02,-9.096682e-02,-1.561903e-02,-1.004196e-01,-3.517472e-02,-5.474177e-02,-2.245669e-02,-1.814844e-02
50%,3.138800e-04,-1.645579e-01,-1.660753e-01,-1.679358e-01,-1.010855e-01,-9.984496e-02,-9.572186e-02,-5.625907e-01,-1.373037e-01,6.801157e-01,...,-2.851295e-03,-5.977947e-02,-2.386234e-02,-9.096682e-02,-1.561903e-02,-1.004196e-01,-3.517472e-02,-5.474177e-02,-2.245669e-02,-1.814844e-02
75%,8.656937e-01,-1.645579e-01,-1.660753e-01,-1.679358e-01,-1.010855e-01,-9.984496e-02,-9.572186e-02,-5.625907e-01,-1.373037e-01,6.801157e-01,...,-2.851295e-03,-5.977947e-02,-2.386234e-02,-9.096682e-02,-1.561903e-02,-1.004196e-01,-3.517472e-02,-5.474177e-02,-2.245669e-02,-1.814844e-02
max,1.731852e+00,2.282308e+01,2.277458e+01,2.332451e+01,3.753263e+01,3.772602e+01,3.875883e+01,1.777491e+00,7.283127e+00,6.801157e-01,...,3.507178e+02,1.672815e+01,4.190704e+01,1.099302e+01,6.402447e+01,9.958219e+00,2.842951e+01,1.826759e+01,4.453016e+01,5.510114e+01


In [108]:
X_test_scaled.head()

,SK_ID_CURR,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_Desconocido,...,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_19,FLAG_DOCUMENT_18,FLAG_DOCUMENT_17,FLAG_DOCUMENT_16,FLAG_DOCUMENT_15,FLAG_DOCUMENT_14,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
0,-0.512450,NaN,NaN,NaN,NaN,NaN,NaN,-0.562591,-0.137304,-1.470338,...,-0.002851,-0.059779,-0.023862,-0.090967,-0.015619,-0.10042,-0.035175,-0.054742,-0.022457,-0.018148
1,0.792989,NaN,NaN,NaN,NaN,NaN,NaN,-0.562591,-0.137304,-1.470338,...,-0.002851,-0.059779,-0.023862,-0.090967,-0.015619,-0.10042,-0.035175,-0.054742,-0.022457,-0.018148
2,-0.485157,-0.573164,-0.566693,-0.534332,-0.101085,-0.099845,-0.095722,1.777491,-0.137304,-1.470338,...,-0.002851,-0.059779,-0.023862,10.993019,-0.015619,-0.10042,-0.035175,-0.054742,-0.022457,-0.018148
3,0.058163,NaN,NaN,NaN,NaN,NaN,NaN,-0.562591,-0.137304,-1.470338,...,-0.002851,-0.059779,-0.023862,-0.090967,-0.015619,-0.10042,-0.035175,-0.054742,-0.022457,-0.018148
4,1.037921,0.276925,0.288427,0.342145,0.188694,0.195197,0.207344,1.777491,-0.137304,-1.470338,...,-0.002851,-0.059779,-0.023862,-0.090967,-0.015619,-0.10042,-0.035175,-0.054742,-0.022457,-0.018148
